---
---
---
# ***Text Preprocessing***
---
---
---

<br>


---
## ***1. Integer Encoding***
---

- *각 단어에 고유한 정수를 mapping*
- *주로 빈도수가 높은 기준으로 정렬하여 mapping*

<br>
<br>

#### 1. ***Integer Encoding***

`method`
>1. *Dictionary* <br>
2. *Counter (collections)* <br>
3. *FreqDist (nltk)*

In [2]:
# Dictionary 사용하기
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# sample text
text = "A barber is a person. a barber is good person. \
a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. \
Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. \
But keeping and keeping such a huge secret to himself was driving the barber crazy. \
the barber went up a huge mountain."

# sentence tokenize
text = sent_tokenize(text)

# text cleaning and word tokenize
vocab = {} 
sentences = []
stop_words = set(stopwords.words('english'))

# frequency count
for i in text:
    sentence = word_tokenize(i) 
    result = []

    for word in sentence: 
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
            if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0 
                vocab[word] += 1
    sentences.append(result) 
    
print('frequency count : ', vocab)


# 빈도수가 2이상인 단어를 추출 후 빈도수가 많은 단어순으로 정렬
vocab_filter = list(filter(lambda x:x[1]>1, sorted(vocab.items(), key=lambda x:x[1], reverse=True)))

# integer index mapping
word_index = {}
for i in range(len(vocab_filter)):
    word_index[vocab_filter[i][0]] = i+1
print('integer index mapping : ', word_index)   

# 상위 5개 단어만 추출
word_index = dict(filter(lambda x:x[1]<=5, word_index.items()))
print('integer index mapping top 5 : ', word_index)  

# dictionary에 존재하지 않는 단어에 번호를 부여하기 위해 out of vocabulary 정보 입력
word_index['OOV'] = len(word_index) + 1

encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_index[w])
        except:
            temp.append(word_index['OOV'])
    encoded.append(temp)
print('result = ', encoded)

frequency count :  {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}
integer index mapping :  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}
integer index mapping top 5 :  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
result =  [[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


In [3]:
# Counter 사용하기
from collections import Counter
print('sentence sample 3 : ',sentences[:3])

# words = np.hstack(sentences)
words = sum(sentences, [])

# frequency count
vocab = Counter(words)
print('frequency count : ', vocab)

# 상위 5개 단어만 추출
# word_index = dict(filter(lambda x:x[1]<=5, word_index.items()))
vocab = vocab.most_common(5)
print('frequency count top 5 : ', vocab)  

# integer index mapping
word_index = {}
for i in range(len(vocab)):
    word_index[vocab[i][0]] = i+1
print('integer index mapping : ', word_index)   

sentence sample 3 :  [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person']]
frequency count :  Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})
frequency count top 5 :  [('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]
integer index mapping :  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [4]:
# FreqDist 사용하기
from nltk import FreqDist

# frequency count
vocab = FreqDist(words)
print('frequency count : ', vocab)

# 상위 5개 단어만 추출
vocab = vocab.most_common(5)
print('frequency count top 5 : ', vocab)  

# integer index mapping
word_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print('integer index mapping : ', word_index)   

frequency count :  <FreqDist with 13 samples and 36 outcomes>
frequency count top 5 :  [('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]
integer index mapping :  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


<br>

#### 2. ***Keras integer mapping***

 - keras tokenizer 함수에는 word count, word index, sequence to index mapping처리를 한번에 할 수 있는 기능이 존재
 - top n개의 단어만 mapping을 하고 싶을때 파라미터로 지정은 가능하지만 sentence to index mapping시에만 적용됨
 - dictionary에서도 top n개의 단어만을 남기고 싶다면 조건에 부합하지 않는 key값들을 제거해야함
 - OOV 단어 보존 시 기존에는 OOV가 마지막 정수로 mapping되었다면 keras에서는 첫번째 정수로 mapping됨

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
print('sentence sample 3 : ',sentences[:3])

# frequency count
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print('frequency count : ', tokenizer.word_counts)

# 상위 5개 단어만 추출
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1)
tokenizer.fit_on_texts(sentences)
# print('frequency count top 5 : ', tokenizer.word_counts)

# integer index mapping
# print('integer index mapping : ', tokenizer.word_index) 
print('result = ', tokenizer.texts_to_sequences(sentences))



# dictionary에서도 top n개의 단어만을 남기고 싶다면 조건에 부합하지 않는 key값들을 제거해야함
'''
vocab_size = 5
words_frequency = [w for w,c in tokenizer.word_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del tokenizer.word_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[w] # 해당 단어에 대한 카운트 정보를 삭제
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))
'''

# Out of Vocabulary 보존하기
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))
print('result = ', tokenizer.texts_to_sequences(sentences))

sentence sample 3 :  [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person']]
frequency count :  OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])
result =  [[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]
단어 OOV의 인덱스 : 1
result =  [[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


---
## ***2. Padding***
---

- *렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업*

<br>
<br>

#### 1. ***Numpy Padding***
> zero padding

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
print('sentence sample 3 : ',sentences[:3])

# index mapping
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
encoded = tokenizer.texts_to_sequences(sentences)
max_len = max(len(item) for item in encoded)
print('max length of sentences = ', max_len)

# padding
for item in encoded: # 각 문장에 대해서
    while len(item) < max_len:   # max_len보다 작으면
        item.append(0)
        
padding_np = np.array(encoded)
print(padding_np)

sentence sample 3 :  [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person']]
max length of sentences =  7
[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


<br>

#### 2. ***Keras Padding***
> back and forth <br>
zero or value 

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = tokenizer.texts_to_sequences(sentences)

# forth zero padding
padded = pad_sequences(encoded)
print('***forth zero padding***')
print(padded)

# back zero padding
padded = pad_sequences(encoded, padding = 'post')
print('***back zero padding***')
print(padded)

# back zero padding and maxlen change
padded = pad_sequences(encoded, padding = 'post', maxlen = 5)
print('***back zero padding and maxlen change***')
print(padded)

# back value padding
last_value = len(tokenizer.word_index) + 1
padded = pad_sequences(encoded, padding = 'post', value = last_value)
print('***back value padding***')
print(padded)

***forth zero padding***
[[ 0  0  0  0  0  1  5]
 [ 0  0  0  0  1  8  5]
 [ 0  0  0  0  1  3  5]
 [ 0  0  0  0  0  9  2]
 [ 0  0  0  2  4  3  2]
 [ 0  0  0  0  0  3  2]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  2]
 [ 7  7  3  2 10  1 11]
 [ 0  0  0  1 12  3 13]]
***back zero padding***
[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]
***back zero padding and maxlen change***
[[ 1  5  0  0  0]
 [ 1  8  5  0  0]
 [ 1  3  5  0  0]
 [ 9  2  0  0  0]
 [ 2  4  3  2  0]
 [ 3  2  0  0  0]
 [ 1  4  6  0  0]
 [ 1  4  6  0  0]
 [ 1  4  2  0  0]
 [ 3  2 10  1 11]
 [ 1 12  3 13  0]]
***back value padding***
[[ 1  5 14 14 14 14 14]
 [ 1  8  5 14 14 14 14]
 [ 1  3  5 14 14 14 14]
 [ 9  2 14 14 14 14 14]
 [ 2  4  3  2 14 14 14]
 [ 3  2 14 14 14 14 14]
 [ 1  4  6 

---
## ***3. One Hot Encoding***
---


> *unique한 단어수가 100개가 있고 실제 출현한 단어수가 200개라면*<br>
*원핫인코딩읜 차원은 (200, 100)이됨* <br>
*즉 단어 수가 row, 유니크한 단어 수가 col(벡터의 차원)이 됨*

`method`
1. *Index integer mapping*
2. *One Hot Encoding*


`one hot encoding의 한계`
 - 단어 수가 많아질수록 벡터의 차원이 계속 늘어나 저장 공간 측면에서 매우 비효율적
 - 단어의 유사도를 표현하지 못함

In [11]:
from konlpy.tag import Okt  

# 형태소 분석기를 통한 tokenize
okt=Okt()  
token=okt.morphs("나는 자연어 처리를 배운다")  
print(token)

word2index={}
for voca in token:
     if voca not in word2index.keys():
       word2index[voca]=len(word2index)
print(word2index)

def one_hot_encoding(word, word2index):
   one_hot_vector = [0]*(len(word2index))
   index=word2index[word]
   one_hot_vector[index]=1
   return one_hot_vector

one_hot_encoding("자연어",word2index)

['나', '는', '자연어', '처리', '를', '배운다']
{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


[0, 0, 1, 0, 0, 0]

In [14]:
# keras 사용하기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# word index create
text="나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

# sentence to int
sub_text="점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded=tokenizer.texts_to_sequences([sub_text])[0]
print(encoded)

# one hot encoding
one_hot = to_categorical(encoded)
print(one_hot)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}
[2, 5, 1, 6, 3, 7]
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


---
## ***4. Splitting Data***
---


<br>

#### 1. ***Supervised Learning***

> *sample dataset*

텍스트(메일의 내용)|레이블(스팸 여부)
---|---
당신에게 드리는 마지막 혜택! ...|스팸 메일
내일 뵐 수 있을지 확인 부탁...|정상 메일
...|...
(광고) 멋있어질 수 있는...|스팸 메일


<br>

#### 2. ***Split X, y***
1. zip function
2. DataFrame
3. Numpy

<br>

#### 3. ***Split test data***
 - train_test_split (sklearn) 

---
## ***5. Text Preprocessing Tools for Korean Text***
---


<br>

#### 1. ***PyKoSpacing***
 - 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지
 
 
<br>

#### 2. ***Py-Hanspell***
 - 네이버 한글 맞춤법 검사기를 바탕으로 만들어진 패키지

In [23]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install git+https://github.com/ssut/py-hanspell.git

In [2]:
# from pykospacing import Spacing

# sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. \
# 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'

# new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
# print(new_sent)

# kospacing_sent = Spacing(new_sent)
# print(sent)
# print(kospacing_sent)

In [26]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "

# 맞춤법 검사
spelled_sent = spell_checker.check(sent)
hanspell_sent = spelled_sent.checked
print(hanspell_sent)
      
# 뛰어쓰기
spelled_sent = spell_checker.check(new_sent)
hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지
김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.


<br>

#### 3. ***SOYNLP***
 - 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
 - 비지도 학습으로 단어 토큰화를 하며 데이터에 자주 등장하는 단어들을 단어로 분석
 - 내부적으로 단어 점수 표로 동작하며 이 점수는 응집 확률(cohesion probability)과 브랜칭 엔트로피(branching entropy)를 활용
 